In [15]:
import pandas as pd
import os
import scipy.stats as stats

def add_hline(latex: str, index: int) -> str:
    """
    Adds a horizontal `index` lines before the last line of the table

    Args:
        latex: latex table
        index: index of horizontal line insertion (in lines)
    """
    lines = latex.splitlines()
    lines.insert(len(lines) + index - 6 - 3, r'\midrule')
    return '\n'.join(lines).replace('NaN', '')

def add_hline2(latex: str, index: int) -> str:
    """
    Adds a horizontal `index` lines before the last line of the table

    Args:
        latex: latex table
        index: index of horizontal line insertion (in lines)
    """
    lines = latex.splitlines()
    lines[8] = r"\cmidrule(lr){1-1} \cmidrule(lr){2-5} \cmidrule(lr){2-5} \cmidrule(lr){6-7} \cmidrule(lr){8-10} \cmidrule(lr){11-12}"  
    lines.insert(len(lines) + index - 6 - 3, r'\midrule')
    return '\n'.join(lines).replace('NaN', '')

In [16]:
carpetas = ['GUROBI','CPLEX','MLM','ESGH','LPDH','Versiones']

In [17]:
gur = []
mlm = []
esgh = []
lpdh = []
cplex = []
versiones = []

for c in carpetas:
    archivos = os.listdir(c)
    for ar in archivos: 
        # print(sorted(ar.rstrip(".xlsx").split("_")))
        # print(sorted(ar.rstrip(".xlsx").split("_")))
        tiempo, solver, capacidad, nodos  = sorted(ar.rstrip(".xlsx").split("_"))
        tiempo = int(tiempo)
        capacidad = int(capacidad[1:])
        nodos = int(nodos[1:])
        datos = pd.read_excel(f"{c}/{ar}")
        datos["Q"] = capacidad
        datos["nodos"] = nodos
        datos["tiempos"] = tiempo
        if solver == "GUROBI":
            gur.append(datos)
        if solver == "CPLEX":
            cplex.append(datos)
        elif solver == "MLM":
            mlm.append(datos)
        elif solver == "ESGH":
            esgh.append(datos)
        elif solver == "LPDH":
            lpdh.append(datos)
        elif solver in ["Expv1", "Expv2", "Expv3", "Expv4"]:
            datos["version"] = "Version" + solver[-1]
            versiones.append(datos)
        else: pass
        
gurobis = pd.concat(gur)
cplexs  = pd.concat(cplex)
mlms    = pd.concat(mlm)
esghs   = pd.concat(esgh)
lpdhs   = pd.concat(lpdh)
versiones   = pd.concat(versiones)

In [18]:
gurobis[["LB","UB"]] = gurobis[["LB","UB"]].round(2)
cplexs[["LB","UB"]] = cplexs[["LB","UB"]].round(2)
mlms[["min", "avg"]] = mlms[["min", "avg"]].round(2)

In [19]:
bks = pd.concat((
    mlms.groupby(["name", "Q", "nodos"]).agg(bks = ("min", "min")).reset_index(), 
    gurobis.groupby(["name", "Q", "nodos"]).agg(bks = ("UB", "min")).reset_index(),
    cplexs.groupby(["name", "Q", "nodos"]).agg(bks = ("UB", "min")).reset_index(),
    esghs.groupby(["name", "Q", "nodos"]).agg(bks = ("cost", "min")).reset_index(),
    lpdhs.groupby(["name", "Q", "nodos"]).agg(bks = ("cost", "min")).reset_index(),
    versiones.groupby(["name", "Q", "nodos"]).agg(bks = ("min", "min")).reset_index(),
))

bks = bks.groupby(["name", "Q", "nodos"]).agg(bks = ("bks", "min")).reset_index()

gurobis = gurobis.merge(bks, on = ["name", "Q", "nodos"])
cplexs  = cplexs.merge(bks, on = ["name", "Q", "nodos"] )
mlms    = mlms.merge(bks, on = ["name", "Q", "nodos"])
esghs   = esghs.merge(bks, on = ["name", "Q", "nodos"])
lpdhs   = lpdhs.merge(bks, on = ["name", "Q", "nodos"])
versiones = versiones.merge(bks, on = ["name", "Q", "nodos"])

In [22]:
gurobis["best"] = gurobis["UB"]
cplexs["best"] = cplexs["UB"]
mlms["best"] = mlms["min"]
versiones["best"] = versiones["min"]
lpdhs["best"] = lpdhs["cost"]
esghs["best"] = esghs["cost"]

gurobis["gap_best"]    = (gurobis["UB"] - gurobis["bks"]) / gurobis["bks"]
cplexs["gap_best"]     = (cplexs["UB"] - cplexs["bks"]) / cplexs["bks"]
mlms["gap_best"]       = (mlms["min"] - mlms["bks"]) / mlms["bks"]
mlms["gap_avg"]        = (mlms["avg"] - mlms["bks"]) / mlms["bks"]
versiones["gap_best"]  = (versiones["min"] - versiones["bks"]) / versiones["bks"]
versiones["gap_avg"]   = (versiones["avg"] - versiones["bks"]) / versiones["bks"]
lpdhs["gap"]           = (lpdhs["cost"] - lpdhs["bks"]) / lpdhs["bks"]
esghs["gap"]           = (esghs["cost"] - esghs["bks"]) / esghs["bks"]

gurobis["hit"]      =( gurobis["best"] - gurobis["bks"]).apply(lambda x: 1 if x == 0 else 0) 
mlms["hit"]         =( mlms["best"] - mlms["bks"]).apply(lambda x: 1 if x == 0 else 0) 
versiones["hit"]    =( versiones["best"] - versiones["best"]).apply(lambda x: 1 if x == 0 else 0) 
lpdhs["hit"]        =( lpdhs["best"] - lpdhs["bks"]).apply(lambda x: 1 if x == 0 else 0) 
esghs["hit"]        =( esghs["best"] - esghs["bks"]).apply(lambda x: 1 if x == 0 else 0) 


# Tabla Comparación Solvers

In [24]:
gurobis3600 = gurobis[gurobis.tiempos == 3600]
cplexs3600 = cplexs[cplexs.tiempos == 3600]

gurobis3600["hit"] = gurobis3600.gap.apply(lambda x: 1 if x < 0.0001 else 0)
cplexs3600["hit"] = cplexs3600.gap.apply(lambda x: 1 if x < 0.0001 else 0)

C:\Users\rablo\AppData\Local\Temp\ipykernel_21512\1280445560.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gurobis3600["hit"] = gurobis3600.gap.apply(lambda x: 1 if x < 0.0001 else 0)


In [25]:
t1_gurobi = gurobis3600[gurobis3600.nodos == 100].groupby(["Q"]).agg(
    Gap = ("gap_best", "mean"),
    LB_Gap = ("gap", "mean"),
    Hits = ("hit", "sum"),
    Time = ("t", "mean")).round(2)

t1_cplex = cplexs3600[cplexs3600.nodos == 100].groupby(["Q"]).agg(
    Gap = ("gap_best", "mean"),
    LB_Gap = ("gap", "mean"),
    Hits = ("hit", "sum"),
    Time = ("t", "mean")).round(2)

filas = ["Gap (\%)", "LB Gap (\%)", "Hits", "Time(sec.)"]
t1_gurobi.columns = ["Gurobi_" + f for f in filas]
t1_cplex.columns = ["CPLEX_" + f for f in filas]
t1 = pd.concat((t1_gurobi, t1_cplex), axis = 1)
t1.columns = pd.MultiIndex.from_tuples([c.split("_") for c in t1.columns])
t1 = t1.sort_values("Q", ascending = False)
t1.loc["overall"] = t1.mean().round(2)
#t1.loc["overall"]["Gurobi","Hits"] = 0
#t1.loc["overall"]["CPLEX","Hits"] = 0
#t1.loc["overall"]["Gurobi","Hits"] = t1["Gurobi","Hits"].sum()
#t1.loc["overall"]["CPLEX","Hits"]  = t1["CPLEX","Hits"].sum()
t1.reset_index(inplace = True)
t1.loc[t1["Q"] == 1000, "Q"] = "$\infty$"
t1

Q   Gurobi                                 CPLEX                    \
            Gap (\%) LB Gap (\%)  Hits Time(sec.) Gap (\%) LB Gap (\%)  Hits   
0  $\infty$     0.00        0.02  24.0     261.94     0.03        0.71  13.0   
1        20     0.00        0.50   7.0    1492.59     0.16        1.75   1.0   
2        15     0.02        0.90   1.0    2317.87     0.26        2.56   0.0   
3        10     0.00        1.60   0.0    2962.13     0.00        1.60   0.0   
4         5     0.01        1.32   0.0    3200.20     0.41        3.15   0.0   
5   overall     0.01        0.87   6.4    2046.95     0.17        1.95   2.8   

              
  Time(sec.)  
0    1698.18  
1    2818.16  
2    3202.71  
3    2962.13  
4    3536.29  
5    2843.49

In [26]:
latex1 = t1.to_latex(
    index = False,
    column_format = "c rrrr rrrr",
    multicolumn_format = "c",
    multirow = True, 
    caption =  "Performance of comercial solvers",
    label = "tab:solvers_results",
    position = "H",
    escape = False)

text_latex1 = add_hline(latex1, index = 5)
print(text_latex1)

\begin{table}[H]
\centering
\caption{Performance of comercial solvers}
\label{tab:solvers_results}
\begin{tabular}{c rrrr rrrr}
\toprule
       Q & \multicolumn{4}{c}{Gurobi} & \multicolumn{4}{c}{CPLEX} \\
         & Gap (\%) & LB Gap (\%) & Hits & Time(sec.) & Gap (\%) & LB Gap (\%) & Hits & Time(sec.) \\
\midrule
$\infty$ &     0.00 &        0.02 & 24.0 &     261.94 &     0.03 &        0.71 & 13.0 &    1698.18 \\
      20 &     0.00 &        0.50 &  7.0 &    1492.59 &     0.16 &        1.75 &  1.0 &    2818.16 \\
      15 &     0.02 &        0.90 &  1.0 &    2317.87 &     0.26 &        2.56 &  0.0 &    3202.71 \\
      10 &     0.00 &        1.60 &  0.0 &    2962.13 &     0.00 &        1.60 &  0.0 &    2962.13 \\
       5 &     0.01 &        1.32 &  0.0 &    3200.20 &     0.41 &        3.15 &  0.0 &    3536.29 \\
\midrule
 overall &     0.01 &        0.87 &  6.4 &    2046.95 &     0.17 &        1.95 &  2.8 &    2843.49 \\
\bottomrule
\end{tabular}
\end{table}


C:\Users\rablo\AppData\Local\Temp\ipykernel_21512\472107493.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex1 = t1.to_latex(


In [27]:
with open("latex/solver_results.tex", "a") as file:
    file.write(text_latex1)
    

# Tablas Griegos

In [28]:
tESGH = esghs[esghs.nodos == 100].groupby(["Q"]).agg(
    Best = ("best", "mean"),
    Gap  = ("gap", "mean"),
    Hits = ("hit", "sum")).round(2)

tMLM  = mlms[(mlms.nodos == 100) & (mlms.tiempos == 60)].groupby(["Q"]).agg(
    Best = ("best", "mean"),
    Avg  = ("avg", "mean"),
    Min_gap = ("gap_best", "mean"),
    Avg_gap = ("gap_avg", "mean"),
    Hits = ("hit", "sum")).round(2)

tESGH.columns = ["ESGH_" + f for f in ["Best","Gap (\%)", "BKS Hits"]]
tMLM.columns  = ["3LM (60s)_" + f for f in ["Best", "Avg.", "Min. Gap (\%)","Avg. Gap (\%)", "BKS Hits"]]
t2 = pd.concat((tESGH, tMLM), axis = 1)
t2.columns = pd.MultiIndex.from_tuples([c.split("_") for c in t2.columns])
t2 = t2.sort_values("Q", ascending = False)
t2["Improvement (\%)", "Min."] = 100 * (t2["ESGH", "Best"] - t2["3LM (60s)", "Best"]) / t2["ESGH", "Best"]
t2["Improvement (\%)", "Avg."] = 100 * (t2["ESGH", "Best"] - t2["3LM (60s)", "Avg."]) / t2["ESGH", "Best"]
t2["Improvement (\%)", "Min."] = t2["Improvement (\%)", "Min."].round(2)
t2["Improvement (\%)", "Avg."] = t2["Improvement (\%)", "Avg."].round(2)
t2.loc["overall"] = t2.mean().round(2)
t2.reset_index(inplace = True)
t2.loc[t2["Q"] == 1000, "Q"] = "$\infty$"
t2 = t2.drop([("ESGH", "Best"),("3LM (60s)","Best"), ("3LM (60s)","Avg.")], axis = 1)
t2

Q     ESGH              3LM (60s)                         \
            Gap (\%) BKS Hits Min. Gap (\%) Avg. Gap (\%) BKS Hits   
0  $\infty$     8.69      2.0          0.02          0.11     51.0   
1        20    18.00      0.0          0.11          0.29     40.0   
2        15    18.91      0.0          0.26          0.64     24.0   
3        10    16.37      0.0          0.58          1.14     12.0   
4         5    13.03      0.0          1.20          1.99      7.0   
5   overall    15.00      0.4          0.43          0.83     26.8   

  Improvement (\%)         
              Min.   Avg.  
0             8.20   8.11  
1            15.26  15.09  
2            15.84  15.52  
3            13.64  13.15  
4            10.44   9.72  
5            12.68  12.32

In [29]:
latex2 = t2.to_latex(
    index = False,
    column_format = "c rr rrr rr",
    multicolumn_format = "c",
    multirow = True, 
    caption =  "Performance against ESGH",
    label = "tab:3lm_resuts_kritikos1",
    position = "H",
    escape = False)

text_latex2 = add_hline(latex2, index = 5)

with open("latex/3LM100 - ESGH.tex", "a") as file:
    file.write(text_latex2)

C:\Users\rablo\AppData\Local\Temp\ipykernel_21512\93436199.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex2 = t2.to_latex(


In [30]:
tESGH = esghs[(esghs.nodos == 100) & (esghs.name.str.contains("R1")) ].groupby(["Q"]).agg(
    Best = ("best", "mean"),
    Gap  = ("gap", "mean"),
    Hits = ("hit", "sum")).round(2)

tLPDH = lpdhs[(lpdhs.nodos == 100) & (lpdhs.name.str.contains("R1")) ].groupby(["Q"]).agg(
    Best = ("best", "mean"),
    Gap  = ("gap", "mean"),
    Hits = ("hit", "sum")).round(2)

tMLM  = mlms[(mlms.nodos == 100) & (mlms.tiempos == 60) & (mlms.name.str.contains("R1"))].groupby(["Q"]).agg(
    Best = ("best", "mean"),
    Avg  = ("avg", "mean"),
    Min_gap = ("gap_best", "mean"),
    Avg_gap = ("gap_avg", "mean"),
    Hits = ("hit", "sum")).round(2)

tESGH.columns = ["ESGH_" + f for f in ["Best","Gap (\%)", "BKS Hits"]]
tLPDH.columns = ["LPDH_" + f for f in ["Best","Gap (\%)", "BKS Hits"]]
tMLM.columns  = ["3LM (60s)_" + f for f in ["Best", "Avg.", "Min. Gap (\%)","Avg. Gap (\%)", "BKS Hits"]]
t3 = pd.concat((tESGH, tLPDH, tMLM), axis = 1)
t3.columns = pd.MultiIndex.from_tuples([c.split("_") for c in t3.columns])
t3 = t3.sort_values("Q", ascending = False)
t3["Improvement (\%)", "Min."] = 100 * (t3["LPDH", "Best"] - t3["3LM (60s)", "Best"]) / t3["LPDH", "Best"]
t3["Improvement (\%)", "Avg."] = 100 * (t3["LPDH", "Best"] - t3["3LM (60s)", "Avg."]) / t3["LPDH", "Best"]
t3["Improvement (\%)", "Min."] = t3["Improvement (\%)", "Min."].round(2)
t3["Improvement (\%)", "Avg."] = t3["Improvement (\%)", "Avg."].round(2)
t3.loc["overall"] = t3.mean().round(2)
t3.reset_index(inplace = True)
t3.loc[t3["Q"] == 1000, "Q"] = "$\infty$"
t3 = t3.drop([("ESGH", "Best"),("LPDH", "Best"),("3LM (60s)","Best"), ("3LM (60s)","Avg.")], axis = 1)
t3

Q     ESGH              LPDH              3LM (60s)                \
            Gap (\%) BKS Hits Gap (\%) BKS Hits Min. Gap (\%) Avg. Gap (\%)   
0  $\infty$     9.20      1.0     7.95      1.0          0.02          0.22   
1        20    15.82      0.0    13.51      0.0          0.16          0.38   
2        15    15.91      0.0    13.24      0.0          0.34          0.72   
3        10    13.86      0.0    12.23      0.0          0.31          0.65   
4         5    13.21      0.0    11.63      0.0          0.72          1.44   
5   overall    13.60      0.2    11.71      0.2          0.31          0.68   

           Improvement (\%)         
  BKS Hits             Min.   Avg.  
0     10.0             7.32   7.14  
1      7.0            11.66  11.47  
2      3.0            11.31  10.98  
3      3.0            10.62  10.33  
4      2.0             9.87   9.23  
5      5.0            10.16   9.83

In [31]:
latex3 = t3.to_latex(
    index = False,
    column_format = "c rr rr rrr rr",
    multicolumn_format = "c",
    multirow = True, 
    caption =  "Performance against LPDH",
    label = "tab:3lm_resuts_kritikos2",
    position = "H",
    escape = False)

text_latex3 = add_hline(latex3, index = 5)

with open("latex/3LM100 - LPDH.tex", "a") as file:
    file.write(text_latex3)

C:\Users\rablo\AppData\Local\Temp\ipykernel_21512\3488436063.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex3 = t3.to_latex(


# Tablas comparativas con gurobi

In [32]:
def hacer_tabla(n,t):
    tGurobi3600 = gurobis3600[gurobis3600.nodos == n].groupby(["Q"]).agg(
        Gap    = ("gap_best", "mean"),
        LB_Gap = ("gap", "mean"),
        Time   = ("t", "mean"),
        Hits = ("hit", "sum")).round(2)

    tGurobi = gurobis[(gurobis.nodos == n) & (gurobis.tiempos == t) ].groupby(["Q"]).agg(
        Best = ("best", "mean"),
        Gap  = ("gap_best", "mean"),
        Hits = ("hit", "sum")).round(2)

    tMLM  = mlms[(mlms.nodos == n) & (mlms.tiempos == t)].groupby(["Q"]).agg(
        Best = ("best", "mean"),
        Avg  = ("avg", "mean"),
        Min_gap = ("gap_best", "mean"),
        Avg_gap = ("gap_avg", "mean"),
        Hits = ("hit", "sum")).round(2)
    
    tGurobi3600.columns = ["Gurobi (3600s)_" + f for f in ["Gap (\%)", "LB Gap (\%)", "Time (sec)", "Hits"]]
    tGurobi.columns = [f"Gurobi ({t}s)_" + f for f in ["Best","Gap (\%)", "BKS Hits"]]
    tMLM.columns  = [f"3LM ({t}s)_" + f for f in ["Best", "Avg.", "Min. Gap (\%)","Avg. Gap (\%)", "BKS Hits"]]
    table = pd.concat((tGurobi3600, tGurobi, tMLM), axis = 1)
    table.columns = pd.MultiIndex.from_tuples([c.split("_") for c in table.columns])
    table = table.sort_values("Q", ascending = False)
    table["Improvement (\%)", "Min."] = 100 * (table[f"Gurobi ({t}s)", "Best"] - table[f"3LM ({t}s)", "Best"]) / table[f"Gurobi ({t}s)", "Best"]
    table["Improvement (\%)", "Avg."] = 100 * (table[f"Gurobi ({t}s)", "Best"] - table[f"3LM ({t}s)", "Avg."]) / table[f"Gurobi ({t}s)", "Best"]
    table["Improvement (\%)", "Min."] = table["Improvement (\%)", "Min."].round(2)
    table["Improvement (\%)", "Avg."] = table["Improvement (\%)", "Avg."].round(2)
    table.loc["overall"] = table.mean().round(2)
    table.reset_index(inplace = True)
    table.loc[table["Q"] == 1000, "Q"] = "$\infty$"
    table = table.drop([(f"Gurobi ({t}s)","Best"), (f"3LM ({t}s)", "Best"), (f"3LM ({t}s)", "Avg.")], axis = 1)
    size = size = "small" if n == 100 else "medium" if n == 150 else "large"
    latex = table.to_latex(
        index = False,
        column_format = "c rrrr rr rrr rr",
        multicolumn_format = "c",
        multirow = True, 
        caption =  f"Performance against Gurobi on {size} instances in {t} seconds",
        label = f"tab:3lm_resuts{n}T{t}",
        position = "H",
        escape = False)
    
    text_latex = add_hline2(latex, index = 5)
    with open(f"latex/3LM{n}T{t}.tex", "a") as file:
        file.write(text_latex)
    
    return text_latex

In [33]:
print(hacer_tabla(100,60))

\begin{table}[H]
\centering
\caption{Performance against Gurobi on small instances in 60 seconds}
\label{tab:3lm_resuts100T60}
\begin{tabular}{c rrrr rr rrr rr}
\toprule
       Q & \multicolumn{4}{c}{Gurobi (3600s)} & \multicolumn{2}{c}{Gurobi (60s)} & \multicolumn{3}{c}{3LM (60s)} & \multicolumn{2}{c}{Improvement (\%)} \\
         &       Gap (\%) & LB Gap (\%) & Time (sec) & Hits &     Gap (\%) & BKS Hits & Min. Gap (\%) & Avg. Gap (\%) & BKS Hits &             Min. &  Avg. \\
\cmidrule(lr){1-1} \cmidrule(lr){2-5} \cmidrule(lr){2-5} \cmidrule(lr){6-7} \cmidrule(lr){8-10} \cmidrule(lr){11-12}
$\infty$ &           0.00 &        0.02 &     261.94 & 24.0 &         0.14 &     41.0 &          0.02 &          0.11 &     51.0 &             0.12 &  0.03 \\
      20 &           0.00 &        0.50 &    1492.59 &  7.0 &         0.47 &     23.0 &          0.11 &          0.29 &     40.0 &             0.38 &  0.19 \\
      15 &           0.02 &        0.90 &    2317.87 &  1.0 &         0.57 &     

C:\Users\rablo\AppData\Local\Temp\ipykernel_21512\2385040745.py:35: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex = table.to_latex(


In [34]:
print(hacer_tabla(150,60))

C:\Users\rablo\AppData\Local\Temp\ipykernel_21512\2385040745.py:35: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex = table.to_latex(


\begin{table}[H]
\centering
\caption{Performance against Gurobi on medium instances in 60 seconds}
\label{tab:3lm_resuts150T60}
\begin{tabular}{c rrrr rr rrr rr}
\toprule
       Q & \multicolumn{4}{c}{Gurobi (3600s)} & \multicolumn{2}{c}{Gurobi (60s)} & \multicolumn{3}{c}{3LM (60s)} & \multicolumn{2}{c}{Improvement (\%)} \\
         &       Gap (\%) & LB Gap (\%) & Time (sec) & Hits &     Gap (\%) & BKS Hits & Min. Gap (\%) & Avg. Gap (\%) & BKS Hits &             Min. &  Avg. \\
\cmidrule(lr){1-1} \cmidrule(lr){2-5} \cmidrule(lr){2-5} \cmidrule(lr){6-7} \cmidrule(lr){8-10} \cmidrule(lr){11-12}
$\infty$ &           0.00 &        0.15 &     746.76 & 20.0 &         0.61 &     26.0 &          1.06 &          1.81 &     31.0 &            -0.39 & -1.11 \\
      20 &           0.09 &        2.75 &    3255.56 &  0.0 &         1.31 &      8.0 &          0.72 &          1.40 &     14.0 &             0.58 & -0.07 \\
      15 &           0.07 &        3.50 &    3412.33 &  0.0 &         1.86 &    

In [35]:
print(hacer_tabla(200,60))

\begin{table}[H]
\centering
\caption{Performance against Gurobi on large instances in 60 seconds}
\label{tab:3lm_resuts200T60}
\begin{tabular}{c rrrr rr rrr rr}
\toprule
       Q & \multicolumn{4}{c}{Gurobi (3600s)} & \multicolumn{2}{c}{Gurobi (60s)} & \multicolumn{3}{c}{3LM (60s)} & \multicolumn{2}{c}{Improvement (\%)} \\
         &       Gap (\%) & LB Gap (\%) & Time (sec) & Hits &     Gap (\%) & BKS Hits & Min. Gap (\%) & Avg. Gap (\%) & BKS Hits &             Min. & Avg. \\
\cmidrule(lr){1-1} \cmidrule(lr){2-5} \cmidrule(lr){2-5} \cmidrule(lr){6-7} \cmidrule(lr){8-10} \cmidrule(lr){11-12}
$\infty$ &           0.00 &        0.45 &    1650.73 & 12.0 &        20.02 &     11.0 &          5.10 &          7.42 &      9.0 &            11.36 & 9.46 \\
      20 &           0.02 &        4.20 &    3552.94 &  0.0 &        12.00 &      0.0 &          6.30 &          9.32 &      1.0 &             4.75 & 2.09 \\
      15 &           0.09 &        5.28 &    3586.08 &  0.0 &         9.13 &      0.

C:\Users\rablo\AppData\Local\Temp\ipykernel_21512\2385040745.py:35: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex = table.to_latex(


In [36]:
print(hacer_tabla(150,120))

C:\Users\rablo\AppData\Local\Temp\ipykernel_21512\2385040745.py:35: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex = table.to_latex(


\begin{table}[H]
\centering
\caption{Performance against Gurobi on medium instances in 120 seconds}
\label{tab:3lm_resuts150T120}
\begin{tabular}{c rrrr rr rrr rr}
\toprule
       Q & \multicolumn{4}{c}{Gurobi (3600s)} & \multicolumn{2}{c}{Gurobi (120s)} & \multicolumn{3}{c}{3LM (120s)} & \multicolumn{2}{c}{Improvement (\%)} \\
         &       Gap (\%) & LB Gap (\%) & Time (sec) & Hits &      Gap (\%) & BKS Hits & Min. Gap (\%) & Avg. Gap (\%) & BKS Hits &             Min. & Avg. \\
\cmidrule(lr){1-1} \cmidrule(lr){2-5} \cmidrule(lr){2-5} \cmidrule(lr){6-7} \cmidrule(lr){8-10} \cmidrule(lr){11-12}
$\infty$ &           0.00 &        0.15 &     746.76 & 20.0 &          1.41 &     22.0 &          0.56 &          1.03 &     33.0 &             0.86 & 0.42 \\
      20 &           0.09 &        2.75 &    3255.56 &  0.0 &          1.92 &      9.0 &          0.41 &          0.97 &     21.0 &             1.44 & 0.90 \\
      15 &           0.07 &        3.50 &    3412.33 &  0.0 &          2.27 

In [37]:
print(hacer_tabla(200,120))

C:\Users\rablo\AppData\Local\Temp\ipykernel_21512\2385040745.py:35: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex = table.to_latex(


\begin{table}[H]
\centering
\caption{Performance against Gurobi on large instances in 120 seconds}
\label{tab:3lm_resuts200T120}
\begin{tabular}{c rrrr rr rrr rr}
\toprule
       Q & \multicolumn{4}{c}{Gurobi (3600s)} & \multicolumn{2}{c}{Gurobi (120s)} & \multicolumn{3}{c}{3LM (120s)} & \multicolumn{2}{c}{Improvement (\%)} \\
         &       Gap (\%) & LB Gap (\%) & Time (sec) & Hits &      Gap (\%) & BKS Hits & Min. Gap (\%) & Avg. Gap (\%) & BKS Hits &             Min. &  Avg. \\
\cmidrule(lr){1-1} \cmidrule(lr){2-5} \cmidrule(lr){2-5} \cmidrule(lr){6-7} \cmidrule(lr){8-10} \cmidrule(lr){11-12}
$\infty$ &           0.00 &        0.45 &    1650.73 & 12.0 &         16.61 &     10.0 &          3.76 &          6.25 &     12.0 &            10.54 &  8.45 \\
      20 &           0.02 &        4.20 &    3552.94 &  0.0 &         17.79 &      0.0 &          2.18 &          4.36 &      4.0 &            12.83 & 11.01 \\
      15 &           0.09 &        5.28 &    3586.08 &  0.0 &         14.1

In [38]:
print(hacer_tabla(150,180))

C:\Users\rablo\AppData\Local\Temp\ipykernel_21512\2385040745.py:35: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex = table.to_latex(


\begin{table}[H]
\centering
\caption{Performance against Gurobi on medium instances in 180 seconds}
\label{tab:3lm_resuts150T180}
\begin{tabular}{c rrrr rr rrr rr}
\toprule
       Q & \multicolumn{4}{c}{Gurobi (3600s)} & \multicolumn{2}{c}{Gurobi (180s)} & \multicolumn{3}{c}{3LM (180s)} & \multicolumn{2}{c}{Improvement (\%)} \\
         &       Gap (\%) & LB Gap (\%) & Time (sec) & Hits &      Gap (\%) & BKS Hits & Min. Gap (\%) & Avg. Gap (\%) & BKS Hits &             Min. & Avg. \\
\cmidrule(lr){1-1} \cmidrule(lr){2-5} \cmidrule(lr){2-5} \cmidrule(lr){6-7} \cmidrule(lr){8-10} \cmidrule(lr){11-12}
$\infty$ &           0.00 &        0.15 &     746.76 & 20.0 &          0.45 &     25.0 &          0.12 &          0.32 &     44.0 &             0.34 & 0.16 \\
      20 &           0.09 &        2.75 &    3255.56 &  0.0 &          1.69 &     10.0 &          0.27 &          0.77 &     25.0 &             1.38 & 0.89 \\
      15 &           0.07 &        3.50 &    3412.33 &  0.0 &          2.06 

In [39]:
print(hacer_tabla(200,180))

C:\Users\rablo\AppData\Local\Temp\ipykernel_21512\2385040745.py:35: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex = table.to_latex(


\begin{table}[H]
\centering
\caption{Performance against Gurobi on large instances in 180 seconds}
\label{tab:3lm_resuts200T180}
\begin{tabular}{c rrrr rr rrr rr}
\toprule
       Q & \multicolumn{4}{c}{Gurobi (3600s)} & \multicolumn{2}{c}{Gurobi (180s)} & \multicolumn{3}{c}{3LM (180s)} & \multicolumn{2}{c}{Improvement (\%)} \\
         &       Gap (\%) & LB Gap (\%) & Time (sec) & Hits &      Gap (\%) & BKS Hits & Min. Gap (\%) & Avg. Gap (\%) & BKS Hits &             Min. &  Avg. \\
\cmidrule(lr){1-1} \cmidrule(lr){2-5} \cmidrule(lr){2-5} \cmidrule(lr){6-7} \cmidrule(lr){8-10} \cmidrule(lr){11-12}
$\infty$ &           0.00 &        0.45 &    1650.73 & 12.0 &          9.69 &     14.0 &          2.27 &          3.76 &     17.0 &             6.32 &  5.01 \\
      20 &           0.02 &        4.20 &    3552.94 &  0.0 &          6.15 &      0.0 &          1.34 &          2.89 &      6.0 &             4.34 &  2.91 \\
      15 &           0.09 &        5.28 &    3586.08 &  0.0 &          3.6